# Lab 02: Sentiment analysis enhancement

Team members:
- SADI Lina
- TAGZIRT Elissa
- MESSAR Cylia
- KHERROUBI Ilhem
- KEDADSA Islam Chakib

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

## 1. Data preparation


### 1.1. Data lecture

In [19]:
# Load the dataset
data = pd.read_csv("finance_sentiment.csv")
sentences = data['Sentence'].tolist()
labels = data['Sentiment']

### 1.2. Data Preprocessing

In [20]:
# Split the data into training and testing sets first
X_train_sentences, X_test_sentences, y_train_labels, y_test_labels = train_test_split(sentences, labels, test_size=0.3, random_state=0)

In [21]:
# Label encoding and one-hot encoding for training labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_labels)
y_train_one_hot = to_categorical(y_train_encoded)

In [22]:
# Label encoding and one-hot encoding for test labels
y_test_encoded = label_encoder.transform(y_test_labels)
y_test_one_hot = to_categorical(y_test_encoded)

### 1.3. Sentence Representation (Tokenization and BERT Embedding Generation)


In [23]:
# Initialize BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
# Parameters for embedding generation
batch_size = 8
max_length = 64

In [25]:
# Function to generate BERT embeddings
def generate_embeddings(sentences):
    sentence_embeddings = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

        # Generate embeddings without computing gradients
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
            sentence_embeddings.append(embeddings.numpy())

    return np.concatenate(sentence_embeddings, axis=0)

In [26]:
# Generate embeddings only for training data
X_train = generate_embeddings(X_train_sentences)
np.save("X_train_embeddings.npy", X_train)

# Generate embeddings for test data
X_test = generate_embeddings(X_test_sentences)
np.save("X_test_embeddings.npy", X_test)

In [27]:
# Reshape data for GRU model
X_train_reshaped = np.expand_dims(X_train, axis=1)
X_test_reshaped = np.expand_dims(X_test, axis=1)

print("Shape of X_train_reshaped:", X_train_reshaped.shape)
print("Shape of X_test_reshaped:", X_test_reshaped.shape)
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_test_one_hot:", y_test_one_hot.shape)

Shape of X_train_reshaped: (4089, 1, 768)
Shape of X_test_reshaped: (1753, 1, 768)
Shape of y_train_one_hot: (4089, 3)
Shape of y_test_one_hot: (1753, 3)


## 2. Training the model (Bert-based/GRU)

In [28]:
# Define the focal loss function
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (tf.ones_like(y_true) - y_true) * (tf.ones_like(y_true) - y_pred)
        focal_loss_value = -alpha_t * tf.pow(tf.ones_like(y_true) - p_t, gamma) * tf.math.log(p_t)
        return tf.reduce_mean(focal_loss_value)
    return focal_loss_fixed

In [29]:
# Define the input size and number of classes
input_size = X_train.shape[1]
num_classes = y_train_one_hot.shape[1]

In [30]:
# Build the GRU model
gru_model = Sequential([
    GRU(256, return_sequences=True, input_shape=(1, input_size), activation='relu'),
    Dropout(0.3),
    GRU(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:
gru_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=focal_loss(gamma=2.0, alpha=0.25), metrics=['accuracy'])
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 1, 256)              │         787,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 128)                 │         148,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 944,643 (3.60 MB)

 Trainable params: 944,643 (3.60 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = gru_model.fit(X_train_reshaped, y_train_one_hot, validation_data=(X_test_reshaped, y_test_one_hot),
                        epochs=50, batch_size=32, callbacks=[early_stopping])

Epoch 1/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.4898 - loss: 0.8744 - val_accuracy: 0.6047 - val_loss: 0.7434
Epoch 2/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6046 - loss: 0.7065 - val_accuracy: 0.6303 - val_loss: 0.6021
Epoch 3/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6379 - loss: 0.5721 - val_accuracy: 0.6452 - val_loss: 0.4874
Epoch 4/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6577 - loss: 0.4630 - val_accuracy: 0.6594 - val_loss: 0.3943
Epoch 5/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6750 - loss: 0.3740 - val_accuracy: 0.6805 - val_loss: 0.3187
Epoch 6/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6935 - loss: 0.3027 - val_accuracy: 0.6891 - val_loss: 0.2579
Epoch 7/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6952 - loss: 0.2449 - val_accuracy: 0.7091 - val_loss: 0.2092
Epoch 8/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7119 - loss: 0.1991 - val_accu

## 3. Testing the model

Evaluate the GRU model's performance on the test set using accuracy, precision, recall, and F1-score.


In [33]:
import timeit
from sklearn.metrics import classification_report

# Measure the time taken for prediction
t = timeit.default_timer()
y_pred_ft = gru_model.predict(X_test_reshaped)
print(f"GRU Model Prediction Time:", timeit.default_timer() - t)

# Convert predictions to class labels
y_pred_classes_ft = np.argmax(y_pred_ft, axis=1)
y_test_classes = np.argmax(y_test_one_hot, axis=1)

# Print classification report
print(classification_report(y_test_classes, y_pred_classes_ft, target_names=["Négatif (P-)", "Positif (P+)", "Neutre (P0)"]))


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
GRU Model Prediction Time: 1.3389488499997242
              precision    recall  f1-score   support

Négatif (P-)       0.49      0.49      0.49       283
Positif (P+)       0.83      0.78      0.80       940
 Neutre (P0)       0.73      0.79      0.76       530

    accuracy                           0.74      1753
   macro avg       0.68      0.69      0.68      1753
weighted avg       0.74      0.74      0.74      1753

